<a href="https://colab.research.google.com/github/SlyFox579/bdt_postblocks/blob/main/PBA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def run(cmd):
  print('>> {}'.format(cmd))
  !{cmd}
  print('')

run('pip install --upgrade pip')

# Install apache-beam.
run('pip install --quiet apache-beam')

# Copy the input file into the local file system.
run('mkdir -p data')
run('wget https://storage.googleapis.com/bdt-beam/users_v.csv -O data/users.csv')
run('wget https://storage.googleapis.com/bdt-beam/orders_v_2022.csv -O data/orders.csv')

In [ ]:
import apache_beam as beam
with beam.Pipeline() as pipe:
  p = (pipe
     |beam.io.ReadFromText('data/users.csv', skip_header_lines=True)
     |beam.Map(lambda x: x.split(','))
     #|beam.Map(lambda x: x[5])
     |beam.Map(lambda x: x[0] + ';' + x[1] + ';' + x[2] + ';' + x[3]
               + ';' + x[4] + ';' + x[5])
     |beam.Map(lambda x: x.split('-'))
     |beam.Map(lambda x: x[0] + ' , ' + x[1] + ' , ' + x[2])
     |beam.Map(lambda x: x.split('/'))
     |beam.Map(lambda x: x[0] + '-' + x[1] + '-' + x[2])
     #|beam.Map(lambda x: x.split(';'))
     #|beam.Map(print)
     |beam.io.WriteToText('data/users_v.csv')
  )

In [ ]:
with beam.Pipeline() as pipe:
  q = (pipe
     |beam.io.ReadFromText('data/orders.csv', skip_header_lines=True)
     |beam.Map(lambda x: x.split(';'))
     |beam.Map(lambda x: x[1] + ';' + x[0] + ';' + x[2] + ';' + x[3])
     #|beam.Map(lambda x: x.split(';'))
     |beam.io.WriteToText('data/orders_v.csv')
  )

In [ ]:
with beam.Pipeline() as pipe:
  p = (pipe
     |beam.io.ReadFromText('/content/data/orders.csv', skip_header_lines=True)
     |beam.Map(lambda x: x.split(';'))
     |beam.Map(lambda x: (x[1],x))
     |beam.combiners.Count.PerKey()
     #|beam.combiners.Mean.PerKey()
     |beam.io.WriteToText('data/orders_v2.csv')
     |beam.Map(print)
  )

In [ ]:
with beam.Pipeline() as pipe:
  p = (pipe
     |beam.io.ReadFromText('data/orders_v2.csv-00000-of-00001')
     |beam.Map(lambda x: x.split(')'))
     |beam.Map(lambda x: x[0])
     |beam.Map(lambda x: x.split("'"))
     |beam.Map(lambda x: x[1] + x[2])
     |beam.Map(lambda x: x.split(", "))
     |beam.Map(lambda x: x[0] + ',' + x[1])
     #|beam.Map(lambda x: x.split(","))
     #|beam.Map(lambda x: (x[1]))
     #|beam.combiners.Mean.Globally()
     #|beam.combiners.Count.PerKey()
     |beam.io.WriteToText('data/orders_v3.csv')
     #|beam.Map(print)
  )

In [ ]:
from apache_beam import CoGroupByKey, Map
with beam.Pipeline() as p:
    users = (p | "Read orders" >> beam.io.ReadFromText("data/users_v.csv-00000-of-00001")
                | beam.Map(lambda x: x.split(';'))
                |beam.Map(lambda x: x[0] + ',' + x[2])
                | beam.Map(lambda x: x.split(','))
             )
    
    orders = (p |beam.io.ReadFromText("data/orders_v3.csv-00000-of-00001")
               | beam.Map(lambda x: x.split(','))
            )

    x = ({"users": users, "orders": orders} | CoGroupByKey() 
                                            |beam.io.WriteToText('data/combined.csv')
    )


In [ ]:
#average number of orders
with beam.Pipeline() as pipe:
  p = (pipe
     |beam.io.ReadFromText('data/combined.csv-00000-of-00001')
     |beam.Map(lambda x: x.split("'"))
     |beam.Map(lambda x: x[1] + ',' + x[5] + ',' + x[9])
     |beam.Map(lambda x: x.split(","))
     |beam.Map(lambda x: x[1] + ',' + x[2])
     |beam.Map(lambda x: x.split(","))
     |beam.Map(lambda x: (x[0],int(x[1])))
     |beam.combiners.Mean.PerKey()
     |beam.Map(print)
  )

('male', 678.2220381110191)
('female', 678.5173913043478)


In [11]:
#total number of orders

with beam.Pipeline() as pipe:
  p = (pipe
     |beam.io.ReadFromText('data/combined.csv-00000-of-00001')
     |beam.Map(lambda x: x.split("'"))
     |beam.Map(lambda x: x[1] + ',' + x[5] + ',' + x[9])
     |beam.Map(lambda x: x.split(","))
     |beam.Map(lambda x: x[1] + ',' + x[2])
     |beam.Map(lambda x: x.split(","))
     |beam.Map(lambda x: (x[0],int(x[1])))
     |beam.CombinePerKey(sum)
     |beam.Map(print)
  )

('female', 780295)
('male', 818614)


In [13]:
! head -3 /content/data/orders.csv

order_no;user_id;product_list;date_purchased
1000;1887;Cassava;2000-01-01
1001;838;Calabash, Water Spinach;2000-01-01
